In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/gemstone.csv")

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df = df.drop(labels='id',axis=1) 

In [5]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z',
       'price'],
      dtype='object')

In [7]:
## Independent and dependent feature

X = df.drop(labels=['price'], axis=1)
y = df[['price']]

In [8]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [9]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [10]:
## Segregating numerical and categorical variable
categorical_cols = X.select_dtypes(include='object').columns
numericals_cols = X.select_dtypes(exclude='object').columns

In [11]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [12]:
numericals_cols


Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [13]:
df['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [26]:
## Define the custom ranking for each ordinal variable
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI1','SI2','VS2','VS1','VVS2','VVS1','IF']

In [27]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [28]:
## Numerical Pipline

num_pipline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)


## Categorical pipline

cut_pipline = Pipeline(
    steps=[
        ('impoter',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler', StandardScaler())
    ]
)

In [29]:
preprocessor = ColumnTransformer([
    ('num_pipline',num_pipline,numericals_cols),
    ('cat_pipline',cut_pipline,categorical_cols)
])

In [30]:
## train test split

from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [31]:
X_train.shape, X_test.shape

((154858, 9), (38715, 9))

In [32]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [35]:
X_train.head()

,num_pipline__carat,num_pipline__depth,num_pipline__table,num_pipline__x,num_pipline__y,num_pipline__z,cat_pipline__cut,cat_pipline__color,cat_pipline__clarity
0,-1.016395,-0.204317,0.402608,-1.202472,-1.187395,-1.194148,-0.132842,-0.936018,-1.154489
1,0.882396,0.720758,-0.118536,0.985177,0.941823,1.036109,-0.132842,-0.320002,0.088951
2,1.529711,0.350728,-1.160823,1.426308,1.394848,1.441611,0.872563,1.528047,0.088951
3,1.896523,0.073206,0.923751,1.741402,1.711965,1.702290,-0.132842,1.528047,-0.532769
4,0.450852,1.738340,1.444895,0.562052,0.525040,0.703019,-2.143651,0.912031,0.088951


In [36]:
## Model Training 

from sklearn.linear_model import LinearRegression,Lasso, Ridge,ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [41]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)
    return mae, rmse, r2_square

In [47]:
## Train Multiple Model
## Model Evaluation  

models = {
    'LinearRegression' : LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'ElasticNet' : ElasticNet()
}

trained_model_list =[]
model_list = []
r2_list= []

for i in range(len(list(models))):
    model  = list(models.values())[i]
    model.fit(X_train,y_train)


    #make prediction
    y_pred = model.predict(X_test)
    mae, rmse,r2_square = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Training Performance")
    print("RMSE",rmse)
    print("MAE",mae)
    print("R2_Score",r2_square*100)
    r2_list.append(r2_square)
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE 1063.5120856681026
MAE 694.970690886727
R2_Score 93.0006952158382


Lasso
Model Training Performance
RMSE 1063.7544943165592
MAE 695.9125054394584
R2_Score 92.99750411833945


Ridge
Model Training Performance
RMSE 1063.5180136176707
MAE 694.9913899815509
R2_Score 93.00061718825025


ElasticNet
Model Training Performance
RMSE 1558.7023903065779
MAE 1076.1651841790588
R2_Score 84.96524176205524


